# Dangerous Item Detection Model Training.ipynb

In [ ]:
# Install dependencies if needed
# !pip install ultralytics opencv-python matplotlib torch>=2.0

**Imports and Setup**

In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO


**Define Paths**

In [ ]:
# Dataset folder (Change folder as required)
DATA_DIR = "Dangerous_Items"

# Image and label folders
IMG_TRAIN = os.path.join(DATA_DIR, "images/train")
IMG_VAL = os.path.join(DATA_DIR, "images/val")
IMG_TEST = os.path.join(DATA_DIR, "images/test")

LBL_TRAIN = os.path.join(DATA_DIR, "labels/train")
LBL_VAL = os.path.join(DATA_DIR, "labels/val")
LBL_TEST = os.path.join(DATA_DIR, "labels/test")

# Config folder
CONFIG_DIR = "configs"
os.makedirs(CONFIG_DIR, exist_ok=True)


**YAML Configuration (Change directory as you want)**

In [ ]:
yaml_content = """
path: Dangerous_Items

train: images/train
val: images/val
test: images/test

nc: 5
names:
  - machete
  - knife
  - baseball_bat
  - rifle
  - gun
"""

yaml_path = os.path.join(CONFIG_DIR, "Dangerous_Items.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("YAML config created at:", yaml_path)


**Visualize Random Labels**

In [ ]:
def visualize_label(img_dir, lbl_dir):
    img_name = random.choice(os.listdir(img_dir))
    img_path = os.path.join(img_dir, img_name)
    lbl_path = os.path.join(lbl_dir, img_name.replace('.jpg', '.txt'))

    img = cv2.imread(img_path)
    h, w, _ = img.shape

    if os.path.exists(lbl_path):
        with open(lbl_path) as f:
            for line in f:
                cls, xc, yc, bw, bh = map(float, line.split())
                x1 = int((xc - bw/2) * w)
                y1 = int((yc - bh/2) * h)
                x2 = int((xc + bw/2) * w)
                y2 = int((yc + bh/2) * h)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(img_name)
    plt.show()

visualize_label(IMG_TRAIN, LBL_TRAIN)


**Check for Empty Labels**

In [ ]:
def count_empty_labels(lbl_dir):
    empty_count = sum(1 for f in os.listdir(lbl_dir) if os.path.getsize(os.path.join(lbl_dir, f)) == 0)
    return empty_count

print("Empty train labels:", count_empty_labels(LBL_TRAIN))
print("Empty val labels:", count_empty_labels(LBL_VAL))
print("Empty test labels:", count_empty_labels(LBL_TEST))


Empty train labels: 554
Empty val labels: 147
Empty test labels: 125


**Train YOLOv8 Model**

In [ ]:
# Load pretrained YOLOv8 model
model = YOLO("yolov8n.pt")  # YOLOv8 nano, lightweight for example

# Train
model.train(
    data=yaml_path,
    epochs=55,
    imgsz=640,
    batch=16
)


**Evaluate Model**

In [ ]:
# Load best weights
model = YOLO("/runs/detect/train/weights/best.pt")

# Validate on test set
metrics = model.val(
    data=yaml_path,
    imgsz=640,
    split="test"
)

print(metrics)


**Predict on Test Images**

In [ ]:
results = model.predict(
    source=IMG_TEST,
    imgsz=640,
    conf=0.25,  # set confidence threshold
    save=True  # saves predictions in runs/detect/predict
)

# Access a single prediction example
r = results[0]
print("Boxes xyxy:", r.boxes.xyxy)
print("Class indices:", r.boxes.cls)
print("Confidence scores:", r.boxes.conf)


**Visualize Predictions**

In [ ]:
for i, r in enumerate(results[:3]):  # show first 3 predictions
    img = cv2.imread(r.path)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"Prediction {i}")
    plt.show()


**Save Results for Sharing**

In [ ]:
# Create a results folder
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)